In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
warnings.simplefilter(action='ignore')

### 1. Import the scraped data from indeed website and put them all in one dataFrame

In [2]:
data1= pd.read_csv("data_full.csv",na_values = 'None')
data1.shape

(4259, 9)

In [3]:
data2=pd.read_csv('devops_withoutDouble.csv',na_values = 'None')
data2.shape

(1265, 9)

In [4]:
data3=pd.read_csv('software+engineer.csv',na_values = 'None')
data3.shape

(3732, 9)

In [5]:
data4=pd.read_csv('more_data.csv',na_values = 'None')
data4.shape

(1178, 9)

In [6]:
df= pd.concat([data1, data2, data3,data4])

In [8]:
df.duplicated().any()

True

In [9]:
# delete duplicated rows
df= df[~df.duplicated(keep='first')]

In [10]:
df.shape

(6889, 9)

### 2.  SALARY

###### Get the rows which their salary value not NAN

In [11]:
# only with salary data present
df_salary = df.dropna(subset=['Salary'])
df_salary.shape

(786, 9)

In [12]:
# rest of the dataframe without salary info
df = df[~df.Salary.isin(df_salary.Salary)]
df.shape

(6103, 9)

##### Remove non numeric chars from salary

In [13]:
df_salary["Salary"] = df_salary["Salary"].str.replace("\n", "")
df_salary["Salary"] = df_salary["Salary"].str.replace(",", "")
df_salary["Salary"] = df_salary["Salary"].str.replace("€", "")
df_salary["Salary"] = df_salary["Salary"].str.replace("\xa0", "")
df_salary.head(5)

,Title,Location,Date,Company,Rating,Count,Salary,Contract,Description
2,Business Developer – Expert sécurité,Arras (62),il y a 18 jours,Uptoo,"4,5",32 avis,27000 - 55000 par an,27 000 € - 55 000 € par an,À PROPOS :Bienvenue dans notre PME spécialiste...
4,Manager Big Data H/F,Paris (75),Il y a plus de 30 jours,Elitegroup Recruitment,NaN,NaN,55000 - 75000 par an,"Temps plein, CDI",Présentation de l'entreprise:Notre client est ...
7,Consultant(e) Business Intelligence (Power Bi)...,Lille (59),il y a 11 jours,sumit,NaN,NaN,30000 - 40000 par an,"Temps plein, CDI","Chez sumit, nous ne voyons pas nos consultants..."
9,Consultant business intelligence : BI / BO (F/H),Paris (75),Il y a plus de 30 jours,Elitech Conseil,NaN,NaN,2900 - 3400 par mois,"Temps plein, CDI",Elitech recrute en urgence !Un Consultant busi...
10,Business Developer Junior H/F,Toulouse (31),il y a 3 jours,Synapse Développement,NaN,NaN,30000 - 35000 par an,"Temps plein, CDI",Devenez Synapsien !Et si vous rejoigniez une é...


###### Extract the deffirent types of salary 

In [14]:
year_salaries = df_salary[df_salary["Salary"].str.contains("an")]
month_salaries = df_salary[df_salary["Salary"].str.contains("mois")]
day_salaries = df_salary[df_salary["Salary"].str.contains("jour")]
hour_salaries = df_salary[df_salary["Salary"].str.contains("heure")]

In [15]:
print(year_salaries.shape)
print(month_salaries.shape)
print(day_salaries.shape)
print(hour_salaries.shape)

(626, 9)
(97, 9)
(49, 9)
(14, 9)


In [16]:
# removing string values("par an", " par mois", etc. from salary dfs)

year_salaries["Salary"] = year_salaries["Salary"].str.replace(" par an", "")
month_salaries["Salary"] = month_salaries["Salary"].str.replace(" par mois", "")
day_salaries["Salary"] = day_salaries["Salary"].str.replace(" par jour", "")
hour_salaries["Salary"] = hour_salaries["Salary"].str.replace(" par heure", "")

###### Functions to get the average salary (avg_sal), the minimum declared value of salary (min_sal), and the maximum declared value of salary (max_sal)

In [17]:
def avg_sal(sal):
    try:
        splt = sal.split(' - ', 1)
        first = float(splt[0])
        second = float(splt[1])
        return (first + second)/2
    except:
        return float(sal)

In [18]:
def min_sal(sal):
    try:
        splt = sal.split(' - ', 1)
        first = float(splt[0])
        return first
    except:
        return float(sal)

In [19]:
def max_sal(sal):
    try:
        splt = sal.split(' - ', 1)
        second = float(splt[1])
        return second
    except:
        return float(sal)

###### Apply the defined functions on the different types of salary 

In [20]:
# min salary

year_salaries["min_salary"] = year_salaries["Salary"].apply(min_sal)
month_salaries["min_salary"] = month_salaries["Salary"].apply(min_sal)
day_salaries["min_salary"] = day_salaries["Salary"].apply(min_sal)
hour_salaries["min_salary"] = hour_salaries["Salary"].apply(min_sal)

In [21]:
# max salary

year_salaries["max_salary"] = year_salaries["Salary"].apply(max_sal)
month_salaries["max_salary"] = month_salaries["Salary"].apply(max_sal)
day_salaries["max_salary"] = day_salaries["Salary"].apply(max_sal)
hour_salaries["max_salary"] = hour_salaries["Salary"].apply(max_sal)

In [22]:
# average salary

year_salaries["avg_salary"] = year_salaries["Salary"].apply(avg_sal)
month_salaries["avg_salary"] = month_salaries["Salary"].apply(avg_sal)
day_salaries["avg_salary"] = day_salaries["Salary"].apply(avg_sal)
hour_salaries["avg_salary"] = hour_salaries["Salary"].apply(avg_sal)

In [23]:
# converting to yearly salary

month_salaries["min_salary"] = month_salaries["min_salary"] * 12
day_salaries["min_salary"] = day_salaries["min_salary"] * 230
hour_salaries["min_salary"] = hour_salaries["min_salary"] * 1607

In [24]:
month_salaries["max_salary"] = month_salaries["max_salary"] * 12
day_salaries["max_salary"] = day_salaries["max_salary"] * 230
hour_salaries["max_salary"] = hour_salaries["max_salary"] * 1607

In [25]:
month_salaries["avg_salary"] = month_salaries["avg_salary"] * 12
day_salaries["avg_salary"] = day_salaries["avg_salary"] * 230
hour_salaries["avg_salary"] = hour_salaries["avg_salary"] * 1607

In [26]:
month_salaries.head(3)

,Title,Location,Date,Company,Rating,Count,Salary,Contract,Description,min_salary,max_salary,avg_salary
9,Consultant business intelligence : BI / BO (F/H),Paris (75),Il y a plus de 30 jours,Elitech Conseil,NaN,NaN,2900 - 3400,"Temps plein, CDI",Elitech recrute en urgence !Un Consultant busi...,34800.0,40800.0,37800.0
323,ALTERNANCE - Assistant business analyste (H/F),Saint-Ouen (93),il y a 2 jours,OCP REPARTITION,NaN,NaN,924 - 1700,"Temps plein, Apprentissage","OCP, au cœur de la chaine de santéAu service d...",11088.0,20400.0,15744.0
403,Stage Ingénieur d’affaires (F/H),Le Kremlin-Bicêtre (94),Il y a plus de 30 jours,IT link,"3,8",19 avis,1000,Stage,Le posteDans le cadre du développement de notr...,12000.0,12000.0,12000.0


###### Put all the salary dataframes together

In [27]:
df_salary = pd.concat([year_salaries, month_salaries, day_salaries, hour_salaries], sort=False)

In [28]:
df_salary.shape

(786, 12)

In [29]:
#rejoining salary data into main scrape_data df
df = pd.concat([df, df_salary])
df.shape

(6889, 12)

In [30]:
df.drop('Salary',axis=1,inplace=True)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6889 entries, 0 to 1146
Data columns (total 11 columns):
Company        6888 non-null object
Contract       4259 non-null object
Count          3541 non-null object
Date           6889 non-null object
Description    6859 non-null object
Location       6889 non-null object
Rating         3614 non-null object
Title          6889 non-null object
avg_salary     786 non-null float64
max_salary     786 non-null float64
min_salary     786 non-null float64
dtypes: float64(3), object(8)
memory usage: 645.8+ KB


## DATE

In [32]:
# count the number of date that contains the word 'jour'
df['Date'].str.contains("jour").mean()*100

99.81129336623603

In [33]:
# parsing date and extract the date in which the job offer has been published
from datetime import datetime, timedelta

def parse_date(date):   
    try:
        N = int(''.join(filter(lambda x: x.isdigit(), date)))
        date_N_days_ago = datetime.now() - timedelta(days=N)
        date_N_days_ago = date_N_days_ago.strftime('%Y-%m-%d')
        return date_N_days_ago
    except:
        pass 

In [35]:
df['Date'] = df['Date'].apply(parse_date)
df['Date'] = df['Date'].fillna(value=datetime.now().strftime('%Y-%m-%d'))

In [36]:
df['Date'].head(5)

0    2020-05-27
1    2020-05-27
3    2020-05-27
5    2020-05-27
6    2020-05-27
Name: Date, dtype: object

###### Extract the department from location, then delete it from location column

In [37]:
# extract department numbers in location
df['Dept']=df['Location'].str.extract('(\d+)')

In [38]:
# deal with NAN values
df['Dept'] = df['Dept'].fillna(0)
df['Dept']=df['Dept'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6889 entries, 0 to 1146
Data columns (total 12 columns):
Company        6888 non-null object
Contract       4259 non-null object
Count          3541 non-null object
Date           6889 non-null object
Description    6859 non-null object
Location       6889 non-null object
Rating         3614 non-null object
Title          6889 non-null object
avg_salary     786 non-null float64
max_salary     786 non-null float64
min_salary     786 non-null float64
Dept           6889 non-null int32
dtypes: float64(3), int32(1), object(8)
memory usage: 672.8+ KB


In [39]:
# Delete the department number from Location
df['Location'] = df['Location'].str.replace(r"\(.*\)","")

In [40]:
df.head(2)

,Company,Contract,Count,Date,Description,Location,Rating,Title,avg_salary,max_salary,min_salary,Dept
0,Socomec,CDI,32 avis,2020-05-27,Référence de l'annonce : H-SSA-2019-59QUI SOMM...,Benfeld,"3,7",Responsable Business Intelligence (BI) H/F,NaN,NaN,NaN,67
1,BCA Expertise,NaN,NaN,2020-05-27,"Leader français de l'expertise automobile, BCA...",Asnières-sur-Seine,NaN,Data Analyst - Business Intelligence H/F,NaN,NaN,NaN,92


### Get the number of opinions from the count column

In [41]:
df.Count=df.Count.str.extract('(\d+)')

In [44]:
# replace NaN values by zero
df.Count.fillna(0, inplace=True)

In [45]:
df.head(2)

,Company,Contract,Count,Date,Description,Location,Rating,Title,avg_salary,max_salary,min_salary,Dept
0,Socomec,CDI,32,2020-05-27,Référence de l'annonce : H-SSA-2019-59QUI SOMM...,Benfeld,"3,7",Responsable Business Intelligence (BI) H/F,NaN,NaN,NaN,67
1,BCA Expertise,NaN,0,2020-05-27,"Leader français de l'expertise automobile, BCA...",Asnières-sur-Seine,NaN,Data Analyst - Business Intelligence H/F,NaN,NaN,NaN,92


### Get unique job offers from contract 

In [46]:
# delete salary from Contract column
import re
df.Contract = df.Contract.replace(re.compile('^[0-9]+'), np.nan, regex=True)
#df.head(5)

In [47]:
print( len(df.Contract.unique()))

51


In [48]:
# define a dictionary with the most commun job types
dic={'CDI' :['cdi'],'CDD' : ['cdd', 'stage', 'apprentissage', 'contrat pro', 'intérim'],'Freelance' :['freelance','indépendant'],
'Temps partiel':['temps','partiel'],'Temps plein':['temps','plein'],'Type_Notknown':['nan']}
dic

{'CDI': ['cdi'],
 'CDD': ['cdd', 'stage', 'apprentissage', 'contrat pro', 'intérim'],
 'Freelance': ['freelance', 'indépendant'],
 'Temps partiel': ['temps', 'partiel'],
 'Temps plein': ['temps', 'plein'],
 'Type_Notknown': ['nan']}

In [51]:
import nltk
tokenizer = nltk.RegexpTokenizer(r'\w+')
def parseContract(x):
    text=str(x)
    # tokenize the text
    text = tokenizer.tokenize(text.lower())
    # delete not chars
    text=[word for word in text if word.isalpha()]
    # keep words with length more than 2 chars
    text=[word for word in text if len(word)>2]
    return text

In [52]:
df['contractWords']= df.apply(lambda x: parseContract(x.Contract), axis=1)
df['contractWords'].head(5)

0    [cdi]
1    [nan]
3    [cdi]
5    [cdi]
6    [cdi]
Name: contractWords, dtype: object

In [53]:
# this function take x (contract value) and search for its correct job type 
def FindJobType(x,desc):
    # verify if the job type is cdi or temps plein or temps partiel or unkonw type
    if len(desc)<=2:
        # check if the job type is totaly included in the contract
        if all(item in x for item in desc):
            return 1
        else:
            return 0
    # if the job type is cdd, then we verify that there is at least one description of it in the contract
    elif any(w in desc for w in x):
     #   print('ok2')
        return 1
    else: return 0

In [54]:
# example
FindJobType(['pro','temps','partiel'],['temps','partiel'])

1

In [55]:
# This function creates five columns in the dataframe df (keys in dictionary Dec), and assign the value 1 (exist) or 0 if not in the column contract
def DummyJobType(df,Dec):
    for Jtype,desc in Dec.items():
        for index ,row in df.iterrows():
            #df[Jtype] = df[Jtype].astype(int)
            #df.set_value(index,Jtype,FindJobType(row['contractWords'],desc))
            df.at[index, Jtype]=FindJobType(row['contractWords'],desc)
            #df[Jtype] = df[Jtype].astype(int)
    return df 

In [56]:
df=DummyJobType(df,dic)

In [60]:
# delete additional columns
df.drop(['Contract','contractWords'],inplace=True,axis=1)

In [61]:
df.shape

(6889, 17)

In [62]:
df.head(3)

,Company,Count,Date,Description,Location,Rating,Title,avg_salary,max_salary,min_salary,Dept,CDI,CDD,Freelance,Temps partiel,Temps plein,Type_Notknown
0,Socomec,32,2020-05-27,Référence de l'annonce : H-SSA-2019-59QUI SOMM...,Benfeld,"3,7",Responsable Business Intelligence (BI) H/F,NaN,NaN,NaN,67,0.0,0.0,0.0,0.0,0.0,1.0
1,BCA Expertise,0,2020-05-27,"Leader français de l'expertise automobile, BCA...",Asnières-sur-Seine,NaN,Data Analyst - Business Intelligence H/F,NaN,NaN,NaN,92,1.0,0.0,0.0,0.0,1.0,0.0
3,Pôle Lille Métropole,59,2020-05-27,"CONTEXTEAvec 36 000 salariés, Ramsay Santé est...",Lille,"2,9",Directeur du contrôle financier - H/F,NaN,NaN,NaN,59,1.0,0.0,0.0,0.0,1.0,0.0


In [59]:
#newDF=df[['Contract','contractWords', 'CDI', 'CDD', 'Freelance', 'Temps partiel', 'Temps plein', 'Type_Notknown']]
#newDF.to_csv('VerifyContractDummies.csv', index=False)

### Replace the Nan values in Rating by 3

In [63]:
df.Rating.fillna(3, inplace=True)

In [64]:
df.duplicated().any()

True

In [65]:
df= df[~df.duplicated(keep='first')]
df.shape

(6701, 17)

In [66]:
#df.to_csv('Data.csv', index=False)
df.to_csv('scrapedClean.csv', index=False)